In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from keras.utils import np_utils

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [5]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.6 MB/s eta 0:00:00


In [6]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
#from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [7]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [8]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [9]:
Fs = 100

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [11]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/transfer_set_50_subs.csv',header=None)

In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.026093,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.032245,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.004273,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.079087,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143


In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape(116718,3000,1)

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [12]:
from keras.utils import np_utils

In [ ]:
from keras.utils import np_utils
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
subject_482_tst_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [13]:
subject_482_tst_df=pd.read_csv('/content/gdrive/My Drive/sub_482/x_test_sub.csv',header=None)

In [ ]:
subject_482_tst_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,-0.115570,-0.115570,-0.110424,-0.144301,-0.218060,-0.275952,-0.265661,-0.247221,-0.216345,-0.156309,...,0.090269,0.088982,0.078262,0.066254,0.066683,0.071829,0.099274,0.128006,0.128006,2
1,-0.001501,0.005360,-0.018654,-0.033234,-0.028088,-0.041811,-0.048243,-0.054676,-0.057678,-0.021227,...,-0.002359,0.026373,0.025087,0.042669,0.040953,0.019512,0.001501,-0.019941,-0.025515,2
2,-0.217289,-0.202789,-0.189994,-0.138391,-0.105552,-0.050111,0.061199,0.103420,0.173788,0.111097,...,0.042861,-0.011728,0.003625,0.010022,0.015140,-0.043714,-0.021110,-0.042008,-0.064184,0
3,-0.003199,-0.100008,-0.153744,-0.026228,-0.022816,-0.045420,-0.077832,0.006184,-0.007463,-0.088493,...,-0.040302,-0.027508,-0.043287,0.052243,-0.037743,-0.082096,-0.086361,-0.076979,-0.159715,0
4,-0.015140,0.006610,0.049684,0.073993,0.068449,0.046272,0.063331,0.069302,-0.001919,-0.024096,...,0.113229,0.070155,0.035184,0.021110,0.023669,-0.008316,-0.006610,-0.000213,-0.024949,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.085082,0.089773,0.111097,0.126023,0.126023,0.118347,0.124744,0.136685,0.173788,0.195538,...,-0.036037,-0.003625,0.048831,0.037317,0.034331,0.002346,0.039449,0.043287,0.039875,2
610,0.003199,0.078258,0.191274,0.130288,0.138817,0.198524,0.181465,0.239465,0.233068,0.259936,...,0.042434,-0.030066,-0.125597,-0.037743,0.168244,-0.060773,-0.009596,-0.050537,-0.116214,0
611,0.112376,0.032625,0.057361,0.103420,0.048831,-0.019405,0.046699,0.028361,0.078685,0.061626,...,0.018125,0.028361,0.088920,0.008743,-0.042008,0.085935,-0.185303,-0.135406,-0.201936,0
612,0.018225,0.008362,-0.019083,-0.067112,-0.049530,0.017368,0.037523,0.047386,0.026373,-0.008791,...,-0.003216,0.004074,0.006218,-0.018225,-0.025944,-0.013508,0.011793,-0.036665,-0.057249,0


In [14]:
x_sub_set=subject_482_tst_df.drop([3000],axis=1)

In [ ]:
x_sub_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.115570,-0.115570,-0.110424,-0.144301,-0.218060,-0.275952,-0.265661,-0.247221,-0.216345,-0.156309,...,0.083836,0.090269,0.088982,0.078262,0.066254,0.066683,0.071829,0.099274,0.128006,0.128006
1,-0.001501,0.005360,-0.018654,-0.033234,-0.028088,-0.041811,-0.048243,-0.054676,-0.057678,-0.021227,...,-0.026802,-0.002359,0.026373,0.025087,0.042669,0.040953,0.019512,0.001501,-0.019941,-0.025515
2,-0.217289,-0.202789,-0.189994,-0.138391,-0.105552,-0.050111,0.061199,0.103420,0.173788,0.111097,...,0.054375,0.042861,-0.011728,0.003625,0.010022,0.015140,-0.043714,-0.021110,-0.042008,-0.064184
3,-0.003199,-0.100008,-0.153744,-0.026228,-0.022816,-0.045420,-0.077832,0.006184,-0.007463,-0.088493,...,0.019831,-0.040302,-0.027508,-0.043287,0.052243,-0.037743,-0.082096,-0.086361,-0.076979,-0.159715
4,-0.015140,0.006610,0.049684,0.073993,0.068449,0.046272,0.063331,0.069302,-0.001919,-0.024096,...,0.131141,0.113229,0.070155,0.035184,0.021110,0.023669,-0.008316,-0.006610,-0.000213,-0.024949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.085082,0.089773,0.111097,0.126023,0.126023,0.118347,0.124744,0.136685,0.173788,0.195538,...,-0.049258,-0.036037,-0.003625,0.048831,0.037317,0.034331,0.002346,0.039449,0.043287,0.039875
610,0.003199,0.078258,0.191274,0.130288,0.138817,0.198524,0.181465,0.239465,0.233068,0.259936,...,-0.048405,0.042434,-0.030066,-0.125597,-0.037743,0.168244,-0.060773,-0.009596,-0.050537,-0.116214
611,0.112376,0.032625,0.057361,0.103420,0.048831,-0.019405,0.046699,0.028361,0.078685,0.061626,...,0.022816,0.018125,0.028361,0.088920,0.008743,-0.042008,0.085935,-0.185303,-0.135406,-0.201936
612,0.018225,0.008362,-0.019083,-0.067112,-0.049530,0.017368,0.037523,0.047386,0.026373,-0.008791,...,0.018654,-0.003216,0.004074,0.006218,-0.018225,-0.025944,-0.013508,0.011793,-0.036665,-0.057249


In [15]:
x_sub_set_rshp=np.array(x_sub_set).reshape(614,3000,1)

In [ ]:
x_sub_set_rshp.shape

(614, 3000, 1)

In [16]:
y_sub_dt=subject_482_tst_df[3000]

In [17]:
y_sub_dt

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [18]:
y_sub_dt_=np_utils.to_categorical(y_sub_dt)

In [ ]:
y_sub_dt_

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [19]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-w_rg1gem
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-w_rg1gem
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=b8e1929901a0fe89b1fe183acf05922db9cd407fb6d03f75348368b9c246c9cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-dpougcu5/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [20]:
from keras_contrib.layers import CRF

In [21]:
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras_contrib.layers import CRF

WINDOW_SIZE = 100

In [22]:
nclass = 5

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)

    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)

    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)

    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

Checking for individual model accuracies wrt to the test set

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
preds_m1 = loaded_model_shuf1.predict(x_sub_set_rshp)
preds_m1

array([[1.6826890e-04, 2.2831915e-03, 9.9145645e-01, 4.0310603e-03,
        2.0609545e-03],
       [3.0522177e-04, 4.5042909e-03, 9.7390026e-01, 1.7299544e-02,
        3.9907671e-03],
       [9.9972540e-01, 1.1455438e-04, 7.1707836e-05, 7.2387302e-05,
        1.5961119e-05],
       ...,
       [9.9957985e-01, 1.5386795e-04, 2.1660692e-04, 3.3203371e-05,
        1.6519536e-05],
       [5.9426814e-01, 3.3617771e-01, 3.5375632e-02, 3.5186727e-05,
        3.4143377e-02],
       [9.9931479e-01, 5.1836052e-04, 1.2759508e-04, 2.2475864e-05,
        1.6806023e-05]], dtype=float32)

In [ ]:
preds_m2 = loaded_model_shuf2.predict(x_sub_set_rshp)
preds_m2

array([[7.4638439e-05, 2.6835883e-03, 9.9610186e-01, 5.2279327e-04,
        6.1706465e-04],
       [1.0595687e-03, 1.5472770e-02, 9.7229427e-01, 8.7195719e-03,
        2.4538792e-03],
       [9.9981350e-01, 8.4725456e-05, 3.6054673e-05, 5.7995028e-05,
        7.7909335e-06],
       ...,
       [9.9985468e-01, 7.5660930e-05, 3.0663712e-05, 3.2379183e-05,
        6.5548270e-06],
       [4.7677088e-01, 4.4202778e-01, 4.1006967e-02, 1.8843357e-05,
        4.0175583e-02],
       [9.9954933e-01, 3.2886391e-04, 9.0831207e-05, 1.7254089e-05,
        1.3720551e-05]], dtype=float32)

In [ ]:
preds_m3 = loaded_model_shuf3.predict(x_sub_set_rshp)
preds_m3

array([[1.43614394e-04, 1.25342410e-03, 9.88879919e-01, 9.17130429e-03,
        5.51682024e-04],
       [8.28254415e-05, 1.45354180e-03, 9.22943771e-01, 7.44331032e-02,
        1.08678790e-03],
       [9.99636173e-01, 2.53320817e-04, 4.54087822e-05, 4.03486774e-05,
        2.47019634e-05],
       ...,
       [9.99579012e-01, 2.73284124e-04, 9.01297681e-05, 4.43794852e-05,
        1.31896795e-05],
       [3.44299674e-01, 3.36031526e-01, 1.88225389e-01, 1.70522384e-04,
        1.31272882e-01],
       [9.99488950e-01, 4.00299527e-04, 5.02231524e-05, 3.69516893e-05,
        2.34355211e-05]], dtype=float32)

In [ ]:
preds_m1_fin=np.argmax(preds_m1,axis=1)
preds_m2_fin=np.argmax(preds_m2,axis=1)
preds_m3_fin=np.argmax(preds_m3,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m1 = confusion_matrix(y_sub_dt, preds_m1_fin)
matrix_m1.diagonal()/matrix_m1.sum(axis=1)

array([0.94565217, 0.10526316, 0.95      , 0.78688525, 0.78481013])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m2 = confusion_matrix(y_sub_dt, preds_m2_fin)
matrix_m2.diagonal()/matrix_m2.sum(axis=1)

array([0.94202899, 0.15789474, 0.96875   , 0.67213115, 0.74683544])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m3 = confusion_matrix(y_sub_dt, preds_m3_fin)
matrix_m3.diagonal()/matrix_m3.sum(axis=1)

array([0.94565217, 0.15789474, 0.94375   , 0.86885246, 0.74683544])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m1 = f1_score(y_sub_dt, preds_m1_fin, average="macro")
prec_m1 = precision_score(y_sub_dt, preds_m1_fin, average="macro")
recall_m1 = recall_score(y_sub_dt, preds_m1_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m1))
    print("Recall: {}".format(recall_m1))
    print("F1 score: {}".format(f1_m1))

    report_m1 = classification_report(y_sub_dt, preds_m1_fin, digits=6)
    print(report_m1)

Precision: 0.7614094684193694
Recall: 0.7145221408583395
F1 score: 0.7220670674785339
              precision    recall  f1-score   support

           0   0.956044  0.945652  0.950820       276
           1   0.333333  0.105263  0.160000        38
           2   0.752475  0.950000  0.839779       160
           3   0.960000  0.786885  0.864865        61
           4   0.805195  0.784810  0.794872        79

    accuracy                       0.858306       614
   macro avg   0.761409  0.714522  0.722067       614
weighted avg   0.845442  0.858306  0.844336       614



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m2 = f1_score(y_sub_dt, preds_m2_fin, average="macro")
prec_m2 = precision_score(y_sub_dt, preds_m2_fin, average="macro")
recall_m2 = recall_score(y_sub_dt, preds_m2_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m2))
    print("Recall: {}".format(recall_m2))
    print("F1 score: {}".format(f1_m2))

    report_m2 = classification_report(y_sub_dt, preds_m2_fin, digits=6)
    print(report_m2)

Precision: 0.7659932534939032
Recall: 0.6975280625856619
F1 score: 0.718329731552163
              precision    recall  f1-score   support

           0   0.959410  0.942029  0.950640       276
           1   0.272727  0.157895  0.200000        38
           2   0.727700  0.968750  0.831099       160
           3   0.976190  0.672131  0.796117        61
           4   0.893939  0.746835  0.813793        79

    accuracy                       0.848534       614
   macro avg   0.765993  0.697528  0.718330       614
weighted avg   0.849774  0.848534  0.840074       614



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m3 = f1_score(y_sub_dt, preds_m3_fin, average="macro")
prec_m3 = precision_score(y_sub_dt, preds_m3_fin, average="macro")
recall_m3 = recall_score(y_sub_dt, preds_m3_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m3))
    print("Recall: {}".format(recall_m3))
    print("F1 score: {}".format(f1_m3))

    report_m3 = classification_report(y_sub_dt, preds_m3_fin, digits=6)
    print(report_m3)

Precision: 0.7882742966684573
Recall: 0.7325969625619033
F1 score: 0.7401291389322788
              precision    recall  f1-score   support

           0   0.952555  0.945652  0.949091       276
           1   0.500000  0.157895  0.240000        38
           2   0.762626  0.943750  0.843575       160
           3   0.883333  0.868852  0.876033        61
           4   0.842857  0.746835  0.791946        79

    accuracy                       0.863192       614
   macro avg   0.788274  0.732597  0.740129       614
weighted avg   0.854062  0.863192  0.850233       614



In [ ]:
optimizer=Adam(lr=1e-4)

In [23]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [24]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)


    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
m=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
hist_19_cnn = m.fit(x_transfer_set_rshp, y_transfer_, epochs=30)

Epoch 1/30
3648/3648 [==============================] - 30s 5ms/step - loss: 0.8085 - acc: 0.6771
Epoch 2/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.5998 - acc: 0.7499
Epoch 3/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.5480 - acc: 0.7723
Epoch 4/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.5033 - acc: 0.7931
Epoch 5/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.4672 - acc: 0.8103
Epoch 6/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.4375 - acc: 0.8250
Epoch 7/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.4132 - acc: 0.8366
Epoch 8/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.4022 - acc: 0.8409
Epoch 9/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.3890 - acc: 0.8473
Epoch 10/30
3648/3648 [==============================] - 18s 5ms/step - loss: 0.3823 - acc: 0.8486
Epoch 11/30
3648/36

In [ ]:
accuracy_1d_cnn = m.evaluate(x_sub_set_rshp, y_sub_dt_)

20/20 [==============================] - 0s 5ms/step - loss: 0.8016 - acc: 0.8518


In [ ]:
preds_m = m.predict(x_sub_set_rshp)

In [ ]:
preds_m_fin=np.argmax(preds_m,axis=1)

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m = f1_score(y_sub_dt, preds_m_fin, average="macro")
prec_m = precision_score(y_sub_dt, preds_m_fin, average="macro")
recall_m = recall_score(y_sub_dt, preds_m_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m))
    print("Recall: {}".format(recall_m))
    print("F1 score: {}".format(f1_m))

    report = classification_report(y_sub_dt, preds_m_fin, digits=6)
    print(report)

Precision: 0.770752419532508
Recall: 0.6984021504076873
F1 score: 0.6999036861721376
              precision    recall  f1-score   support

           0   0.933566  0.967391  0.950178       276
           1   0.500000  0.078947  0.136364        38
           2   0.756345  0.931250  0.834734       160
           3   0.841270  0.868852  0.854839        61
           4   0.822581  0.645570  0.723404        79

    accuracy                       0.851792       614
   macro avg   0.770752  0.698402  0.699904       614
weighted avg   0.837102  0.851792  0.831079       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m = confusion_matrix(y_sub_dt, preds_m_fin)
matrix_m.diagonal()/matrix_m.sum(axis=1)

array([0.9673913 , 0.07894737, 0.93125   , 0.86885246, 0.64556962])

In [ ]:
cm_m = matrix_m.astype('float') / matrix_m.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_m.diagonal()

array([0.9673913 , 0.07894737, 0.93125   , 0.86885246, 0.64556962])

# light weight model trained on transfer set + subject specific train set and testing on subject specific test set

In [25]:
subject_482_tr_df=pd.read_csv('/content/gdrive/My Drive/sub_482/x_train_sub.csv',header=None)

In [ ]:
subject_482_tr_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,-0.037317,-0.035184,-0.053523,-0.013007,0.056081,0.106405,0.121332,0.129435,0.153744,0.198097,...,-0.088920,-0.118773,-0.130714,-0.128156,-0.092332,-0.072287,-0.056081,-0.107685,-0.138817,2
1,0.022085,-0.017796,-0.064110,-0.042669,-0.010506,0.002359,-0.012222,-0.040524,-0.072258,-0.054247,...,0.119858,0.074831,-0.004503,-0.031090,-0.081263,-0.052103,-0.084265,-0.057249,-0.053389,4
2,-0.127577,-0.109566,-0.066254,-0.008791,-0.037952,-0.106564,-0.128006,-0.106993,-0.100132,-0.077833,...,-0.001501,0.037952,0.036665,0.046528,0.048243,0.011364,0.021656,0.037094,0.021656,4
3,-0.147303,-0.149447,-0.105278,-0.070971,-0.057678,-0.042240,-0.037523,-0.040953,-0.046957,-0.017796,...,0.239931,0.229639,0.205624,0.192759,0.182039,0.137869,0.111710,0.105278,0.122860,2
4,-0.071861,-0.088920,-0.128156,-0.128156,-0.117920,-0.088067,-0.066743,-0.033478,-0.030919,-0.008743,...,0.031772,-0.009596,-0.039875,-0.053523,-0.083802,-0.053096,-0.012581,-0.015140,0.001919,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,-0.002359,0.139156,-0.008791,0.067970,0.106136,0.143015,0.113426,0.005789,0.059822,0.031948,...,-0.274666,-0.113855,-0.155022,0.121145,-0.032377,0.225350,-0.048672,-0.414036,-0.345852,0
2452,-0.136259,-0.109391,-0.123891,-0.130714,-0.159288,-0.149053,-0.161847,-0.102141,-0.091905,-0.026228,...,-0.175068,-0.268039,-0.137538,-0.193406,-0.041581,-0.178906,-0.052243,-0.233495,-0.170376,0
2453,0.016081,0.049101,-0.076117,-0.019083,-0.055534,-0.042669,-0.136583,-0.041811,-0.082550,-0.010078,...,-0.121573,-0.110424,-0.085123,-0.070114,-0.073544,-0.042240,-0.027231,0.014366,-0.018225,0
2454,-0.009220,-0.006647,-0.006218,-0.010506,0.011364,-0.017368,-0.025515,-0.032377,-0.011793,0.010935,...,-0.045242,-0.006218,0.029804,0.075260,0.103991,0.104420,0.073544,0.057678,0.033663,2


In [ ]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [26]:
transfer_df_plus_sub_tr = transfer_df.append(subject_482_tr_df)

In [ ]:
transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,-0.002359,0.139156,-0.008791,0.067970,0.106136,0.143015,0.113426,0.005789,0.059822,0.031948,...,-0.274666,-0.113855,-0.155022,0.121145,-0.032377,0.225350,-0.048672,-0.414036,-0.345852,0
2452,-0.136259,-0.109391,-0.123891,-0.130714,-0.159288,-0.149053,-0.161847,-0.102141,-0.091905,-0.026228,...,-0.175068,-0.268039,-0.137538,-0.193406,-0.041581,-0.178906,-0.052243,-0.233495,-0.170376,0
2453,0.016081,0.049101,-0.076117,-0.019083,-0.055534,-0.042669,-0.136583,-0.041811,-0.082550,-0.010078,...,-0.121573,-0.110424,-0.085123,-0.070114,-0.073544,-0.042240,-0.027231,0.014366,-0.018225,0
2454,-0.009220,-0.006647,-0.006218,-0.010506,0.011364,-0.017368,-0.025515,-0.032377,-0.011793,0.010935,...,-0.045242,-0.006218,0.029804,0.075260,0.103991,0.104420,0.073544,0.057678,0.033663,2


In [27]:
x_transfer_df_plus_sub_tr=transfer_df_plus_sub_tr.drop([3000],axis=1)

In [ ]:
x_transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,-0.002359,0.139156,-0.008791,0.067970,0.106136,0.143015,0.113426,0.005789,0.059822,0.031948,...,-0.052532,-0.274666,-0.113855,-0.155022,0.121145,-0.032377,0.225350,-0.048672,-0.414036,-0.345852
2452,-0.136259,-0.109391,-0.123891,-0.130714,-0.159288,-0.149053,-0.161847,-0.102141,-0.091905,-0.026228,...,-0.323481,-0.175068,-0.268039,-0.137538,-0.193406,-0.041581,-0.178906,-0.052243,-0.233495,-0.170376
2453,0.016081,0.049101,-0.076117,-0.019083,-0.055534,-0.042669,-0.136583,-0.041811,-0.082550,-0.010078,...,-0.097559,-0.121573,-0.110424,-0.085123,-0.070114,-0.073544,-0.042240,-0.027231,0.014366,-0.018225
2454,-0.009220,-0.006647,-0.006218,-0.010506,0.011364,-0.017368,-0.025515,-0.032377,-0.011793,0.010935,...,-0.072258,-0.045242,-0.006218,0.029804,0.075260,0.103991,0.104420,0.073544,0.057678,0.033663


In [28]:
y_transfer_df_plus_sub_tr = transfer_df_plus_sub_tr[3000]

In [ ]:
y_transfer_df_plus_sub_tr

0       4
1       3
2       2
3       2
4       0
       ..
2451    0
2452    0
2453    0
2454    2
2455    2
Name: 3000, Length: 119174, dtype: int64

In [29]:
x_transfer_df_plus_sub_tr_rshp=np.array(x_transfer_df_plus_sub_tr).reshape(119174,3000,1)

In [ ]:
x_transfer_df_plus_sub_tr_rshp.shape

(119174, 3000, 1)

In [30]:
y_transfer_df_plus_sub_tr_=np_utils.to_categorical(y_transfer_df_plus_sub_tr)

In [ ]:
y_transfer_df_plus_sub_tr_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [59]:
light_wt_model=get_model_1d_cnn()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 2995, 4)           28        
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 1497, 4)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 1493, 4)           84        
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 746, 4)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 743, 10)           170 

In [ ]:
hist = light_wt_model.fit(x_transfer_df_plus_sub_tr_rshp, y_transfer_df_plus_sub_tr_, epochs=30)

Epoch 1/30
3725/3725 [==============================] - 18s 4ms/step - loss: 0.7323 - acc: 0.7019
Epoch 2/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.5405 - acc: 0.7764
Epoch 3/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.4939 - acc: 0.7978
Epoch 4/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.4618 - acc: 0.8133
Epoch 5/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.4388 - acc: 0.8238
Epoch 6/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.4194 - acc: 0.8321
Epoch 7/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.3994 - acc: 0.8416
Epoch 8/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.3900 - acc: 0.8444
Epoch 9/30
3725/3725 [==============================] - 16s 4ms/step - loss: 0.3811 - acc: 0.8488
Epoch 10/30
3203/3725 [========================>.....] - ETA: 2s - loss: 0.3736 - acc: 0.8526

In [54]:
accuracy = light_wt_model.evaluate(x_sub_set_rshp, y_sub_dt_)

20/20 [==============================] - 0s 2ms/step - loss: 0.6729 - acc: 0.8518


In [55]:
preds_light_wt_model = light_wt_model.predict(x_sub_set_rshp)

20/20 [==============================] - 0s 2ms/step


In [56]:
preds_light_wt_model_fin=np.argmax(preds_light_wt_model,axis=1)

In [57]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_light_wt_model = f1_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
prec_light_wt_model = precision_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
recall_light_wt_model = recall_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_light_wt_model))
    print("Recall: {}".format(recall_light_wt_model))
    print("F1 score: {}".format(f1_light_wt_model))

    report = classification_report(y_sub_dt, preds_light_wt_model_fin, digits=6)
    print(report)

Precision: 0.7446897017549989
Recall: 0.6954489574464414
F1 score: 0.6953486790585629
              precision    recall  f1-score   support

           0   0.943262  0.963768  0.953405       276
           1   0.333333  0.052632  0.090909        38
           2   0.738916  0.937500  0.826446       160
           3   0.866667  0.852459  0.859504        61
           4   0.841270  0.670886  0.746479        79

    accuracy                       0.851792       614
   macro avg   0.744690  0.695449  0.695349       614
weighted avg   0.831532  0.851792  0.830989       614



In [58]:
from sklearn.metrics import confusion_matrix
matrix_light_wt_model = confusion_matrix(y_sub_dt, preds_light_wt_model_fin)
matrix_light_wt_model.diagonal()/matrix_light_wt_model.sum(axis=1)

array([0.96376812, 0.05263158, 0.9375    , 0.85245902, 0.67088608])

In [ ]:
import keras

In [ ]:
teacher_model=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_sub_set_rshp, y_sub_dt_)

20/20 [==============================] - 12s 8ms/step - loss: 0.8515 - accuracy: 0.8518


In [ ]:
preds_teacher = teacher_model.predict(x_sub_set_rshp)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
print(classification_report(y_sub_dt, preds_teacher_fin, digits=6))

              precision    recall  f1-score   support

           0   0.955882  0.942029  0.948905       276
           1   0.384615  0.131579  0.196078        38
           2   0.738095  0.968750  0.837838       160
           3   0.955556  0.704918  0.811321        61
           4   0.810811  0.759494  0.784314        79

    accuracy                       0.851792       614
   macro avg   0.768992  0.701354  0.715691       614
weighted avg   0.845077  0.851792  0.838525       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.94202899, 0.13157895, 0.96875   , 0.70491803, 0.75949367])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm

array([[0.94202899, 0.02173913, 0.01449275, 0.        , 0.02173913],
       [0.05263158, 0.13157895, 0.63157895, 0.        , 0.18421053],
       [0.        , 0.0125    , 0.96875   , 0.0125    , 0.00625   ],
       [0.        , 0.        , 0.29508197, 0.70491803, 0.        ],
       [0.12658228, 0.        , 0.11392405, 0.        , 0.75949367]])

In [ ]:
cm.diagonal()

array([0.94202899, 0.13157895, 0.96875   , 0.70491803, 0.75949367])

In [ ]:
#student_model=get_model_cnn() #issue with unmatched shape
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_, epochs=40)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/40
3648/3648 [==============================] - 32s 8ms/step - categorical_accuracy: 0.7038 - student_loss: 0.7251 - distillation_loss: 0.0023
Epoch 2/40
3648/3648 [==============================] - 27s 8ms/step - categorical_accuracy: 0.7768 - student_loss: 0.5414 - distillation_loss: 0.0017
Epoch 3/40
3648/3648 [==============================] - 27s 8ms/step - categorical_accuracy: 0.7936 - student_loss: 0.5025 - distillation_loss: 0.0016
Epoch 4/40
3648/3648 [==============================] - 27s 8ms/step - categorical_accuracy: 0.8040 - student_loss: 0.4785 - distillation_loss: 0.0015
Epoch 5/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8116 - student_loss: 0.4651 - distillation_loss: 0.0015
Epoch 6/40
3648/3648 [==============================] - 28s 8ms/step - categorical_accuracy: 0.8144 - student_loss: 0.4552 - distillation_loss: 0.0014
Epoch 7/40
3648/3648 [==============================] - 27s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

20/20 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8436 - student_loss: 0.6529


In [ ]:
preds = student_model.predict(x_sub_set_rshp)

In [ ]:
preds

array([[1.5962323e-10, 1.9023717e-06, 9.9382955e-01, 6.1685271e-03,
        6.4299031e-12],
       [1.9450433e-06, 2.8558824e-07, 9.6445358e-01, 3.5544273e-02,
        1.8657283e-12],
       [9.9999964e-01, 1.5916196e-07, 2.6343776e-07, 4.3370186e-12,
        5.9598340e-09],
       ...,
       [9.9999964e-01, 2.8801589e-08, 3.9198838e-07, 7.9134416e-12,
        4.7927995e-09],
       [5.7105690e-01, 1.8801220e-01, 1.0653605e-02, 2.9304945e-10,
        2.3027731e-01],
       [9.9819332e-01, 4.1593882e-04, 1.6684322e-04, 1.1304857e-09,
        1.2239161e-03]], dtype=float32)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
preds_fin

array([2, 2, 0, 0, 2, 0, 0, 2, 4, 0, 0, 1, 3, 0, 0, 2, 2, 2, 0, 0, 4, 0,
       0, 2, 0, 4, 3, 0, 1, 0, 0, 3, 0, 4, 2, 0, 0, 2, 2, 3, 0, 0, 0, 0,
       2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 4, 0, 2,
       3, 0, 2, 0, 0, 0, 2, 2, 0, 4, 3, 3, 2, 0, 3, 4, 0, 2, 2, 0, 0, 0,
       0, 2, 1, 4, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2,
       2, 3, 2, 4, 2, 3, 0, 3, 3, 0, 3, 0, 0, 3, 2, 3, 0, 0, 0, 0, 3, 0,
       4, 0, 0, 3, 0, 0, 2, 2, 0, 2, 0, 4, 0, 2, 2, 0, 2, 2, 0, 4, 0, 2,
       0, 0, 4, 1, 0, 0, 0, 0, 2, 0, 4, 2, 2, 4, 2, 2, 0, 3, 2, 0, 2, 0,
       0, 2, 3, 2, 2, 0, 4, 3, 0, 2, 0, 0, 3, 0, 0, 2, 2, 0, 1, 0, 2, 2,
       0, 0, 4, 0, 2, 0, 3, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0,
       0, 3, 0, 4, 0, 3, 4, 0, 0, 0, 4, 2, 2, 2, 2, 2, 0, 2, 0, 4, 0, 0,
       0, 4, 4, 2, 2, 2, 0, 0, 0, 0, 4, 2, 0, 2, 4, 0, 3, 4, 0, 0, 0, 0,
       2, 4, 2, 4, 4, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 4, 2, 2, 4,
       3, 2, 0, 0, 3, 0, 4, 2, 2, 1, 0, 0, 2, 0, 0,

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [ ]:
print(accuracy_score(y_sub_dt, preds_fin))
#print(precision_score(y_sub_dt, preds_fin))
#print(recall_score(y_sub_dt, preds_fin))
#print(f1_score(y_sub_dt, preds_fin))

0.8436482084690554


In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_sub_dt, preds_fin, average="macro")
prec = precision_score(y_sub_dt, preds_fin, average="macro")
recall = recall_score(y_sub_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec))
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_sub_dt, preds_fin, digits=6)
    print(report)

Precision: 0.70524693156681
Recall: 0.6961140670217637
F1 score: 0.6882080057690511
              precision    recall  f1-score   support

           0   0.942652  0.952899  0.947748       276
           1   0.222222  0.052632  0.085106        38
           2   0.759162  0.906250  0.826211       160
           3   0.830769  0.885246  0.857143        61
           4   0.771429  0.683544  0.724832        79

    accuracy                       0.843648       614
   macro avg   0.705247  0.696114  0.688208       614
weighted avg   0.817105  0.843648  0.825006       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_stu = confusion_matrix(y_sub_dt, preds_fin)
matrix_stu.diagonal()/matrix_stu.sum(axis=1)

array([0.95289855, 0.05263158, 0.90625   , 0.8852459 , 0.6835443 ])

In [ ]:
cm_stu = matrix_stu.astype('float') / matrix_stu.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_stu.diagonal()

array([0.95289855, 0.05263158, 0.90625   , 0.8852459 , 0.6835443 ])

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

20/20 [==============================] - 0s 2ms/step - categorical_accuracy: 0.8436 - student_loss: 0.6529


In [ ]:
#Distiller's performance (student model's prformance)

#alpha=0.9,
#temperature=4

#30 epochs 85.57
#40 epochs 84.85
#50 epochs 83.88
#60 epochs 83.88

#alpha=0.9,
#temperature=5

#30 epochs 85.18
#40 epochs 85.41
#50 epochs 84.01
#60 epochs